# Data Preperation for modelling in Pytorch

In [1]:
import pandas as pd
import numpy as np
import json
import pickle
from pathlib import Path
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

In [3]:
EXPERIMENT_NAME = "fs_06_load_calendar_future_weather"
DATA_DIR = Path("../../data/processed")
OUTPUT_DIR= DATA_DIR / "ml_data" / EXPERIMENT_NAME

SCALERS_DIR = OUTPUT_DIR / "scalers"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
SCALERS_DIR.mkdir(exist_ok=True)

In [4]:
# Load data
df = pd.read_parquet(DATA_DIR / "data_combined.parquet")

In [5]:
y = df["actual_load_mw"].to_frame()
X = df

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.33, shuffle=False)


X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

## Intermediate Outlier check
Checking for outliers is important to help choose the most appropriate scaler.

In [15]:
def iqr_outlier_stats(X):
    stats = {}
    for col in X.columns:
        q1 = X[col].quantile(0.25)
        q3 = X[col].quantile(0.75)
        iqr = q3 - q1
        if iqr == 0:
            stats[col] = 0.0
            continue
        mask = (X[col] < q1 - 1.5 * iqr) | (X[col] > q3 + 1.5 * iqr)
        stats[col] = mask.mean()
    return pd.Series(stats).sort_values(ascending=False)

outlier_frac = iqr_outlier_stats(X_train.select_dtypes(include=np.number))
outlier_frac


tp_future            0.104052
tp                   0.103907
ssrd_future          0.051929
ssrd                 0.051905
wind_speed_future    0.024811
wind_speed           0.024714
actual_load_mw       0.000000
t2m                  0.000000
t2m_future           0.000000
dtype: float64

In [49]:
outlier_frac = iqr_outlier_stats(y_train)
outlier_frac

actual_load_mw    0.0
dtype: float64

Seems like the total precipitation has a 10% outlier rate. This would suggest using a robust scaler.

## Scale Features

In [16]:
X_scaler = RobustScaler()
y_scaler = RobustScaler()

# Fit scalers on training data only
X_train_scaled = X_scaler.fit_transform(X_train)
y_train_scaled = y_scaler.fit_transform(y_train).squeeze()

# Transform validation and test sets using fitted scalers
X_val_scaled = X_scaler.transform(X_val)
X_test_scaled = X_scaler.transform(X_test)

y_val_scaled = y_scaler.transform(y_val).squeeze()
y_test_scaled = y_scaler.transform(y_test).squeeze()

print(f"Scaled shapes - Train: {X_train_scaled.shape}, Val: {X_val_scaled.shape}, Test: {X_test_scaled.shape}")

Scaled shapes - Train: (164763, 11), Val: (40576, 11), Test: (40577, 11)


## Windowing

Create sequences for time series forecasting


In [51]:
def create_sequences(X, y, window_size=4*24*7, forecast_horizon=4*24):
    """
    Create sequences for time series forecasting.
    
    Args:
        X: Feature array (n_samples, n_features)
        y: Target array (n_samples,)
        window_size: Number of time steps to look back
        forecast_horizon: Number of time steps to forecast ahead
        
    Returns:
        X_seq: Array of shape (n_sequences, window_size, n_features)
        y_seq: Array of shape (n_sequences, forecast_horizon)
    """
    n_samples = len(X)
    
    X_seq = []
    y_seq = []
    
    for i in range(n_samples - window_size - forecast_horizon + 1):
        X_seq.append(X[i:i+window_size])
        y_seq.append(y[i+window_size:i+window_size+forecast_horizon])
    
    return np.array(X_seq), np.array(y_seq)

# Configuration
WINDOW_SIZE = 4*24*7  # Look back the last 7 days
FORECAST_HORIZON = 4*24  # Forecast 96 steps ahead

# Create sequences for each split
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, WINDOW_SIZE, FORECAST_HORIZON)
X_val_seq, y_val_seq = create_sequences(X_val_scaled, y_val_scaled, WINDOW_SIZE, FORECAST_HORIZON)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, WINDOW_SIZE, FORECAST_HORIZON)

print(f"Train sequences: {X_train_seq.shape} -> {y_train_seq.shape}")
print(f"Val sequences:   {X_val_seq.shape} -> {y_val_seq.shape}")
print(f"Test sequences:  {X_test_seq.shape} -> {y_test_seq.shape}")


Train sequences: (164278, 672, 5) -> (164278, 96)
Val sequences:   (39879, 672, 5) -> (39879, 96)
Test sequences:  (39879, 672, 5) -> (39879, 96)


## Save Data and Artifacts


In [ ]:
# Save numpy arrays
np.save(OUTPUT_DIR / "X_train.npy", X_train_seq)
np.save(OUTPUT_DIR / "y_train.npy", y_train_seq)
np.save(OUTPUT_DIR / "X_val.npy", X_val_seq)
np.save(OUTPUT_DIR / "y_val.npy", y_val_seq)
np.save(OUTPUT_DIR / "X_test.npy", X_test_seq)
np.save(OUTPUT_DIR / "y_test.npy", y_test_seq)

# Save scalers
with open(SCALERS_DIR / "X_scaler.pkl", "wb") as f:
    pickle.dump(X_scaler, f)
with open(SCALERS_DIR / "y_scaler.pkl", "wb") as f:
    pickle.dump(y_scaler, f)

# Save metadata
meta = {
    "window_size": WINDOW_SIZE,
    "forecast_horizon": FORECAST_HORIZON,
    "n_features": X_train_scaled.shape[1],
    "feature_names": list(X.columns),
    "train_samples": len(X_train_seq),
    "val_samples": len(X_val_seq),
    "test_samples": len(X_test_seq),
    "train_date_range": {
        "start": str(X_train.index.min()),
        "end": str(X_train.index.max())
    },
    "val_date_range": {
        "start": str(X_val.index.min()),
        "end": str(X_val.index.max())
    },
    "test_date_range": {
        "start": str(X_test.index.min()),
        "end": str(X_test.index.max())
    },
    "scaler_type": "RobustScaler",
    "split_ratio": {
        "train": 0.7,
        "val": 0.15,
        "test": 0.15
    }
}

with open(OUTPUT_DIR / "meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print(f"Saved data to {OUTPUT_DIR}")


Saved data to ..\..\data\processed\ml_data\fs_002_weather_and_load
